<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/065_Reversible_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ♻️ Pattern 1: Reversible Actions

When agents interact with the real world, **mistakes are inevitable** — even if the agent makes a good decision with the info it has, the outside world might throw a curveball.

### 🔄 Why Reversible Actions Matter

Reversible actions give your agent a **safety net** — a way to “undo” operations when things go wrong. But to do that, you need to **capture enough context at the time of execution** to allow for a full reversal later.

> 🧠 Reversibility isn't always simple. Deleting a calendar event? Easy to undo.
> But reversing a read email? That might just mean sending a follow-up correction — and that’s okay.

---

### 🧪 Code Example: ReversibleAction Class




In [ ]:
class ReversibleAction:
    def __init__(self, execute_func, reverse_func):
        self.execute = execute_func
        self.reverse = reverse_func
        self.execution_record = None

    def run(self, **args):
        """Execute action and record how to reverse it."""
        result = self.execute(**args)
        self.execution_record = {
            "args": args,
            "result": result,
            "timestamp": datetime.now().isoformat()
        }
        return result

    def undo(self):
        """Reverse the action using recorded information."""
        if not self.execution_record:
            raise ValueError("No action to reverse")
        return self.reverse(**self.execution_record)

# Example using reversible actions
create_event = ReversibleAction(
    execute_func=calendar.create_event,
    reverse_func=lambda **record: calendar.delete_event(record["result"]["event_id"])
)

send_invite = ReversibleAction(
    execute_func=calendar.send_invite,
    reverse_func=lambda **record: calendar.cancel_invite(record["result"]["invite_id"])
)

This is a powerful pattern, and there are **a few key elements** you should focus on to understand both how it works and why it matters in the context of agent design:

---

### 🔁 1. **Explicit Separation of Execution and Reversal**

```python
self.execute = execute_func
self.reverse = reverse_func
```

* The class is designed around **two functions**:

  * One that does something (e.g., schedule a meeting)
  * One that undoes that thing (e.g., delete the meeting)
* This is what enables **reversible logic** — the core idea of this pattern.

---

### 📦 2. **Recording Execution Details for Reversal**

```python
self.execution_record = {
    "args": args,
    "result": result,
    "timestamp": datetime.now().isoformat()
}
```

* To reverse an action, you need to remember **what was done**, **with what inputs**, and **what the outcome was**.
* The `execution_record` is like a mini audit trail — capturing everything you might need to undo an action later.

> 🧠 This is the core of “reversibility”: no action is reversible unless you *remember* how it happened.

---

### 🧨 3. **Fail-Safe Undo**

```python
if not self.execution_record:
    raise ValueError("No action to reverse")
```

* This guards against calling `undo()` on an action that hasn’t been run.
* It's a small line, but it reflects good defensive programming — **don’t assume the system is in a valid state**.

---

### 💡 4. **Real-World Reversibility Example**

```python
create_event = ReversibleAction(
    execute_func=calendar.create_event,
    reverse_func=lambda **record: calendar.delete_event(record["result"]["event_id"])
)
```

* You’re showing a **real-world use case**: creating a calendar event can be reversed by deleting it.
* This makes the abstract class *concretely useful* in agent workflows.

---

### ✅ Summary: What to Take Away

| 🔍 Focus Area                               | 💬 Why It Matters                                             |
| ------------------------------------------- | ------------------------------------------------------------- |
| Clear separation of `execute` and `reverse` | Makes your system robust and modular                          |
| Tracking execution metadata                 | Enables accurate, context-aware reversal                      |
| Defensive programming (`undo` checks)       | Prevents cascading errors                                     |
| Real-world mapping                          | Shows how AI agents can safely interact with external systems |




## ♟️ Pattern 2: Transaction Management

In the world of agents, **real-world tasks are rarely just one-step operations**. Think about scheduling a meeting — it’s not just “create an event.” You might need to:

* ✅ Check everyone’s availability
* 📅 Create a calendar event
* 📢 Send out invitations or notifications

These are **interconnected actions**, and if any one of them fails, the whole process can break.

---

### 🔄 Enter: Transaction Management

The **transaction pattern** allows us to group related steps into a **single coordinated operation** — so they either all succeed, or they all fail and get rolled back.

This ensures your system doesn’t end up in a broken or inconsistent state.

> 💡 **Analogy:** It’s like a “mission abort” switch — if anything goes wrong, you can rewind everything to a clean state.

---

### 🚨 Why It Matters

When combined with **reversible actions**, transaction management becomes a safety net for your agents:

* 🧹 Automatically roll back earlier steps if later ones fail
* ✅ Maintain consistency across systems (no orphaned invites, no ghost calendar entries)
* 🤖 Prevent weird bugs from half-completed agent workflows

---

### 📌 Example Scenarios

* **Meeting scheduling:** You don’t want invites sent if the calendar event wasn’t created.
* **Purchase approval:** Don’t deduct budget unless all approvals are in place.
* **Multi-agent workflows:** One agent’s failure shouldn’t leave the system in limbo.



In [ ]:
class ActionTransaction:
    def __init__(self):
        self.actions = []
        self.executed = []
        self.committed = False
        self.transaction_id = str(uuid.uuid4())

    def add(self, action: ReversibleAction, **args):
        """Queue an action for execution."""
        if self.committed:
            raise ValueError("Transaction already committed")
        self.actions.append((action, args))

    async def execute(self):
        """Execute all actions in the transaction."""
        try:
            for action, args in self.actions:
                result = action.run(**args)
                self.executed.append(action)
        except Exception as e:
            # If any action fails, reverse everything done so far
            await self.rollback()
            raise e

    async def rollback(self):
        """Reverse all executed actions in reverse order."""
        for action in reversed(self.executed):
            await action.undo()
        self.executed = []

    def commit(self):
        """Mark transaction as committed."""
        self.committed = True

This is a **powerful abstraction** for safely coordinating multiple reversible actions — a transaction manager for your agent workflows. Here's what you should focus on and why it matters:

---

## 🔍 Key Concepts to Pay Attention To

### 1. **Transaction as a Unit of Work**

```python
self.actions = []
self.executed = []
```

* `actions` = the plan (what will be done)
* `executed` = the history (what has been done so far)
* This structure separates **intended** actions from **completed** ones, which is essential for rollback safety.

---

### 2. **Safe Additions Until Committed**

```python
if self.committed:
    raise ValueError("Transaction already committed")
```

* Once a transaction is finalized (`committed`), no new actions can be added.
* This protects against unexpected side effects and bugs that come from modifying a “sealed” process.

---

### 3. **All-or-Nothing Execution**

```python
for action, args in self.actions:
    result = action.run(**args)
    self.executed.append(action)
```

* Actions are executed **one at a time**.
* If **any action fails**, it jumps to rollback — nothing is left half-finished.

---

### 4. **Automatic Rollback on Failure**

```python
except Exception as e:
    await self.rollback()
    raise e
```

* This is huge. If even one step fails, the system automatically:

  * Rewinds all previously completed steps
  * Leaves the environment in a clean state

> 💡 Think: a safe “undo chain” that guarantees consistency.

---

### 5. **Reverse Order Rollback**

```python
for action in reversed(self.executed):
    await action.undo()
```

* Rollbacks happen in **reverse order** — this mirrors the classic “stack unwinding” from exception handling in traditional programming.

---

### 6. **Commit = Final Lock-In**

```python
self.committed = True
```

* When you're confident everything went well and no further changes are needed, you **commit** the transaction.
* After this, the transaction is immutable — this prevents accidental or malicious modification.

---

## 🧠 Why This Pattern Matters for Agents

| 🔧 Feature       | 💥 Why It’s Important                        |
| ---------------- | -------------------------------------------- |
| Reversibility    | Prevents long-term damage from AI mistakes   |
| Isolation        | Ensures clean execution without side effects |
| Rollback support | Enables safe retries and recovery strategies |
| Atomicity        | You get either a full win or a clean fail    |
| Debuggability    | Clear execution logs and failure points      |

---

## ✅ Summary: What You’re Building

You’re not just running agent tools —
**You’re creating resilient, auditable, real-world-safe AI workflows.**


